In [2]:
import pandas as pd
import numpy as np 
import tensorflow as tf 
import matplotlib.pyplot as plt
import copy
import re

In [3]:
data = pd.read_csv('/home/kuba/Downloads/train.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


`Extracting titles to estimate age of NaN`

In [4]:
def normalize(x): 
    
    out = (x - x.mean())/x.std()
    return out, x.mean(), x.std()

In [5]:
def get_nice_data(data):
    
    x = copy.copy(data)
    
    # sex to numeric
    x = np.array(x)
    x[:,1] = x[:,1] == 'male'
    # converting to binary
    x[:,1] = x[:,1] * 1
    
    # normalize age
    a = x[:,2]
    
    for i in range(a.size): 
        a[i] = np.nan_to_num(a[i])
    
    k = np.where(a != 0)
    a[k] = normalize(a[k])[0]
    a[a == 0] = normalize(a[k])[1]
    
    x[:,2] = a
    
    # normalize Fare 
    x[:,5] = np.array(x[:,5], dtype = float)
    a = x[:,5]
    for i in range(a.size): 
        a[i] = np.nan_to_num(a[i])
    x[:,5] = normalize(a)[0]
    
    # normalize Cabin (0 if nan)
    
    a = x[:,6]
    for i in range(a.size): 
        a[i] = np.nan_to_num(a[i])
    
    k = np.where(a != 0)
    a[k] = 1 
    a = np.array(a, dtype = int)
    x[:,6] = a
    
    # to classes Embarked
    
    b = x[:,7]
    for i in range(b.size): 
        b[i] = np.nan_to_num(b[i])
    b[b == 'S'] = 1
    b[b == 'C'] = 2
    b[b == 'Q'] = 3

    b = np.array(b,dtype = int)
    x[:,7] = b
    
    return np.array(x, dtype = float)

In [6]:
# getting only interesting ones

train = data[['Pclass','Sex','Age','SibSp','Parch','Fare','Cabin','Embarked']]
test = data[['Survived']]

x_train = get_nice_data(train) 
x_test = np.array(test)
x_test = x_test.flatten()

In [11]:
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Dense(8, activation = tf.nn.relu),
                                    tf.keras.layers.Dense(2,  activation = tf.nn.softmax)])

In [12]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [19]:
model.fit(x_train, x_test, epochs=1000 , verbose=0) 

In [20]:
# not splitting data to get maximum training
# testing on kaggle
model.evaluate(x_train,x_test)

891/891 [==============================] - 0s 81us/sample - loss: 0.3746 - acc: 0.8485


[0.37461829690553256, 0.8484849]

In [21]:
kaggle = pd.read_csv('/home/kuba/wakacje/machine_lerning/test.csv')
Id = np.array(kaggle['PassengerId'])

In [22]:
topredict = kaggle[['Pclass','Sex','Age','SibSp','Parch','Fare','Cabin','Embarked']]
topredict = get_nice_data(topredict)

In [24]:
out = model.predict(topredict)
out_ = np.argmax(out, axis = 1)
Out_file = pd.DataFrame({'PassengerId': Id, 'Survived': out_})
Out_file.to_csv('Titanic_Kaggle', index=False)

`it got us 77% of accuracy on Kaggle - we are overfitting data, lets come up with more observations`

In [25]:
# lets double our samples by adding some noise to data
temp= copy.copy(x_train)

# we will work on 2 and 5 th columns 
to_add = normalize(temp[:,2])[1]
temp[:,2] += to_add 

to_add = normalize(temp[:,5])[1]
temp[:,5] += to_add

x2_train = np.vstack([x_train, temp])
x2_test  = np.vstack([x_test, x_test]).flatten()


x2_train.shape, x2_test.shape


((1782, 8), (1782,))

In [26]:
model.fit(x2_train, x2_test, epochs=10)

Epoch 1/10
1782/1782 [==============================] - 0s 232us/sample - loss: 0.3774 - acc: 0.8457
Epoch 2/10
1782/1782 [==============================] - 0s 151us/sample - loss: 0.3769 - acc: 0.8462
Epoch 3/10
1782/1782 [==============================] - 0s 44us/sample - loss: 0.3764 - acc: 0.8468
Epoch 4/10
1782/1782 [==============================] - 0s 43us/sample - loss: 0.3767 - acc: 0.8462
Epoch 5/10
1782/1782 [==============================] - 0s 44us/sample - loss: 0.3770 - acc: 0.8462
Epoch 6/10
1782/1782 [==============================] - 0s 53us/sample - loss: 0.3771 - acc: 0.8440
Epoch 7/10
1782/1782 [==============================] - 0s 46us/sample - loss: 0.3765 - acc: 0.8485
Epoch 8/10
1782/1782 [==============================] - 0s 48us/sample - loss: 0.3764 - acc: 0.8462
Epoch 9/10
1782/1782 [==============================] - 0s 53us/sample - loss: 0.3762 - acc: 0.8496
Epoch 10/10
1782/1782 [==============================] - 0s 51us/sample - loss: 0.3761 - acc: 0.84

In [27]:
model.evaluate(x_train,x_test)

891/891 [==============================] - 0s 160us/sample - loss: 0.3746 - acc: 0.8485


[0.3745767464474529, 0.8484849]

In [29]:
out = model.predict(topredict)
out_ = np.argmax(out, axis = 1)
Out_file = pd.DataFrame({'PassengerId': Id, 'Survived': out_})
Out_file.to_csv('Titanic_Kaggle', index=False)

`We got 77% again, for now on i tried extracting data from titles of people and for NaNs in age I assigned average age per title, but that worsend my accuracy`  